In [16]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [46]:
path = "/gender_detector_data/"

import pickle

data_vectorizer = pickle.load(open(path + "vec_vectorizer", "rb"))

print (data_vectorizer.shape)

(30000, 34011)


In [47]:
def read_labels():
    labels = []
    with open(path + "labels") as f:
        for i in f.readlines():
            s = i.replace("\n", "")
            labels.append(s)

    return labels

labels = [int(i) for i in read_labels()]

In [75]:
#data = data_vectorizer

data = np.loadtxt(path + "features_w2v_full", delimiter=',')

In [58]:
print (data.shape)

(30000, 300)


In [76]:
from sklearn.model_selection import train_test_split
X = data
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#X_train = X_train.todense()
#X_test = X_test.todense()

In [65]:
def keras_perceptron_plain(X, y):

    tf.reset_default_graph()
    sess = tf.InteractiveSession()

    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=len(X[0])))
    #model.add(Dense(18, activation='relu', input_dim=400))
    #model.add(Dropout(0.2))
    #model.add(Dense(23, activation='relu', input_dim=400))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(X, y, epochs=3, batch_size=32)

    return model

In [77]:
model = keras_perceptron_plain(X_train, y_train)

scores = model.evaluate(X_test, y_test, verbose=0)

print (scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
20100/20100 [==============================] - 2s - loss: 0.6902 - acc: 0.5308       

In [67]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_accuracy(pred, y_test):
    count_corr = 0
    for i in zip(y_test, pred):
        if i[0] == i[1]:
            count_corr += 1
        
    return count_corr, (count_corr/len(y_test))

def get_metrics(predicted, correct):
    count_corr, accuracy = get_accuracy(predicted, y_test)
    print ("all: " + str(len(predicted)) + " " + "correct: " + str(count_corr))
    print ("accuracy" + str(accuracy))
    print ("precision " + str(precision_score(correct, predicted)))
    print ("recall " + str(recall_score(correct, predicted)))
    print ("f1 " + str(f1_score(correct, predicted)))

In [78]:
labels_pred = model.predict(X_test)

In [79]:
labels_pred_new = []

for i in labels_pred:
    proba_m = i[0]
    proba_f = 1 - i[0]
    
    if proba_m > proba_f:
        labels_pred_new.append(0)
    else:
        labels_pred_new.append(1)
        
get_metrics(labels_pred_new, y_test)

all: 9900 correct: 4510
accuracy0.45555555555555555
precision 0.457291112294
recall 0.723883740522
f1 0.560502283105
